In [1]:
import time

import numpy as np
from circuit import Circuit
from mps_stuff import ising_impo, circuit_imps

from scipy.optimize import minimize

In [2]:
d = 2
chimax = 2
J = 1
g = 0.5

In [3]:
# treat the cavity as a qubit and do an arbitrary X rotation
def cav_xrot(params):
    theta = params[0]
    return (0, -theta/2)
# treat the cavity as a qubit and do an arbitrary Y rotation
def cav_yrot(params):
    theta = params[0]
    return (theta/2, 0)
# treat the cavity as a qubit and rotate from Z to X
def cav_zx(params): return (np.pi/4, 0)
# treat the cavity as a qubit and rotate from X to Z
def cav_xz(params): return (-np.pi/4, 0)
# treat the cavity as a qubit and rotate from Z to Y
def cav_zy(params): return (0, -np.pi/4)
# treat the cavity as a qubit and rotate from Y to Z
def cav_yz(params): return (0, np.pi/4)
# rotate qubit from Z to X
def qub_zx(params): return (np.pi/2, np.pi/2, np.pi/2)
# rotate qubit from X to Z
def qub_xz(params): return (np.pi/2, np.pi/2, -np.pi/2)
# rotate qubit from Z to Y
def qub_zy(params): return (np.pi/2, 0, np.pi/2)
# rotate qubit from Y to Z
def qub_yz(params): return (np.pi/2, 0, -np.pi/2)
# SNAP equivalent of exp(i theta ZZ)
def snap_zz(params):
    theta = params[0]
    return [theta, -theta]

c = Circuit([("qubit", "p", d), ("cavity", "b", chimax)])

# arbitrary one-qubit rotations
c.add_gate("rotation")
c.add_gate("displacement", n_params=1, fn=cav_xrot)
c.add_gate("displacement", n_params=1, fn=cav_yrot)
c.add_gate("displacement", n_params=1, fn=cav_xrot)

# XX rotation
c.add_gate("rotation", n_params=0, fn=qub_xz)
c.add_gate("displacement", n_params=0, fn=cav_xz)
c.add_gate("snap", n_params=1, fn=snap_zz)
c.add_gate("rotation", n_params=0, fn=qub_zx)
c.add_gate("displacement", n_params=0, fn=cav_zx)

# YY rotation
c.add_gate("rotation", n_params=0, fn=qub_yz)
c.add_gate("displacement", n_params=0, fn=cav_yz)
c.add_gate("snap", n_params=1, fn=snap_zz)
c.add_gate("rotation", n_params=0, fn=qub_zy)
c.add_gate("displacement", n_params=0, fn=cav_zy)

# ZZ rotation
c.add_gate("snap", n_params=1, fn=snap_zz)

# arbitrary one-qubit rotations
c.add_gate("rotation")
c.add_gate("displacement", n_params=1, fn=cav_xrot)
c.add_gate("displacement", n_params=1, fn=cav_yrot)
c.add_gate("displacement", n_params=1, fn=cav_xrot)

c.assemble()

In [4]:
def energy(params, circuit, Hamiltonian, psi):
    psi = circuit_imps(params, circuit)
    E = (Hamiltonian.expectation_value(psi)).real
    return E

rng = np.random.default_rng() 
params = rng.uniform(high=2*np.pi, size=c.n_params)
psi = circuit_imps(params, c)
print("norm of wave function = {0}".format(psi.norm))
print()

Hamiltonian = ising_impo(J, g)
t1 = time.time()
result = minimize(energy, x0=params, args=(c, Hamiltonian, psi), method='nelder-mead')
sweet_spot = result.x
t2 = time.time()

print("goddamn, that took {}s to optimize".format(t2-t1))
print("sweet spot = {}".format(sweet_spot))
print("num function evaluations: {}".format(result['nfev']))
print("num iterations: {}".format(result['nit']))
print("termination msg: {}".format(result['message']))

norm of wave function = 1.0

goddamn, that took 97.01415967941284s to optimize
sweet spot = [5.62331355 1.12856283 2.85698406 6.06494089 2.96633052 0.14973327
 5.79001977 1.70583356 0.67554644 4.00034017 3.37894867 1.35377546
 2.54913671 6.2909282  2.64125927]
num function evaluations: 3000
num iterations: 2227
termination msg: Maximum number of function evaluations has been exceeded.


In [5]:
from scipy.integrate import quad

def infinite_gs_energy(J, g):
    """
    Straight from tenpy docs: https://tenpy.readthedocs.io/en/latest/intro/examples/tfi_exact.html
    """
    def f(k, lambda_):
        return np.sqrt(1 + lambda_**2 + 2 * lambda_ * np.cos(k))

    E0_exact = -g / (J * 2. * np.pi) * quad(f, -np.pi, np.pi, args=(J / g, ))[0]
    return E0_exact

holo_gs = circuit_imps(sweet_spot, c)
holo_E = energy(sweet_spot, c, Hamiltonian, holo_gs)
E0 = infinite_gs_energy(J, g)
print("holoMPS: E = {:.8}".format(holo_E))
print(" theory: E = {:.8}".format(E0))
print("  error: {:.7%}".format(np.abs(1 - holo_E/E0)))

holoMPS: E = -1.0630585
 theory: E = -1.0635444
  error: 0.0456925%
